In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb


In [2]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


def c_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, g_kdl)


def c_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    c_u2c(q_none, qdot_none)
    
def c_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    c_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot)
    
    
def c_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.NonlinearEffects(rbdlmodel, q_np, qdot_np, c_rbdl)


def g_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, g_kdl)


def g_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_u2c(q_none)
    
def g_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot, qddot)
    
def g_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, g_rbdl)
    
    
def id_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    id_u2c(q_none, qdot_none, qddot_none)
    
    
def id_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2 
    id_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot_none)
    
    
def id_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, id_rbdl)

    
def m_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToMass(q_kdl, m_kdl)


def m_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    m_u2c(q_none)
    
def m_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    m = pb.calculateMassMatrix(pbmodel, q_none)
    
    
def m_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.CompositeRigidBodyAlgorithm(rbdlmodel, q_np, m_rbdl)
    
    
def aba_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_u2c(q_none, qdot_none, tau_none)
    
    
def crba_u2c_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_u2c(q_none, qdot_none, tau_none)
    
    
def fd_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.ForwardDynamics(rbdlmodel, q_np, qdot_np, tau_np, qddot_np)
    

In [3]:
#pbmodel = pb.loadURDF("pantilt.urdf")
sim = pb.connect(pb.DIRECT)
#pbmodel = pb.loadURDF("/urdf4timing/1dof.urdf")
ndofs = 10
urdf_nr = list(range(ndofs + 1))
urdf_nr.pop(0)


nitr = 1000
nrepeat = 100
    

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/urdf/snake_robot.urdf"
root = 'base_link'
tip = "link16"

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)

rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
asd = u2c.URDFparser()
robot_desc = asd.from_file(path_to_urdf)

jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = asd.get_n_joints(root, tip)

gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
gravity_u2c = [0, 0, -9.81]

#rbdl declarations
q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
g_rbdl = np.zeros(njoints)
c_rbdl = np.zeros(njoints)
id_rbdl = np.zeros(njoints)

#kdl declatations
q_kdl = kdl.JntArray(njoints)
qdot_kdl = kdl.JntArray(njoints)
g_kdl = kdl.JntArray(njoints)

#u2c and pybullet declarations
q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints
g_pb = [None]*njoints

qddot = []
for i in range(njoints):
    qddot.append(0.)

qdot = []
for i in range(njoints):
    qdot.append(0.)
    
    
m_kdl = kdl.JntSpaceInertiaMatrix(njoints)
m_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
m_rbdl = np.zeros(m_rbdl)
tau_np = np.zeros(njoints)
tau_none = [None]*njoints

g_u2c = asd.get_gravity_rnea(root, tip, gravity_u2c)
c_u2c = asd.get_coriolis_rnea(root, tip)
m_u2c = asd.get_inertia_matrix_crba(root, tip)
fd_aba_u2c = asd.get_forward_dynamics_aba(root, tip, gravity_u2c)
fd_crba_u2c = asd.get_forward_dynamics_crba(root, tip, gravity_u2c)
id_u2c = asd.get_inverse_dynamics_rnea(root, tip, gravity_u2c)




/home/lmjohann/urdf2casadi/examples/urdf/snake_robot.urdf


In [4]:
timeit_g_u2c = repeat("g_u2c_func()", setup = "from __main__ import g_u2c_func", repeat = nrepeat, number = nitr)
mediantime_g_u2c  = median(timeit_g_u2c)/nitr*1000000

timeit_g_pb = repeat("g_pb_func()", setup = "from __main__ import g_pb_func", repeat = nrepeat, number = nitr)
mediantime_g_pb = median(timeit_g_pb)/nitr*1000000

timeit_g_rbdl = repeat("g_rbdl_func()", setup = "from __main__ import g_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_g_rbdl = median(timeit_g_rbdl)/nitr*1000000

timeit_g_kdl = repeat("g_kdl_func()", setup = "from __main__ import g_kdl_func", repeat = nrepeat, number = nitr)
mediantime_g_kdl = median(timeit_g_kdl)/nitr*1000000


timeit_c_u2c = repeat("c_u2c_func()", setup = "from __main__ import c_u2c_func", repeat = nrepeat, number = nitr)
mediantime_c_u2c  = median(timeit_c_u2c)/nitr*1000000

timeit_c_pb = repeat("c_pb_func()", setup = "from __main__ import c_pb_func", repeat = nrepeat, number = nitr)
mediantime_c_pb = median(timeit_c_pb)/nitr*1000000

timeit_c_rbdl = repeat("c_rbdl_func()", setup = "from __main__ import c_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_c_rbdl = median(timeit_c_rbdl)/nitr*1000000

timeit_c_kdl = repeat("c_kdl_func()", setup = "from __main__ import c_kdl_func", repeat = nrepeat, number = nitr)
mediantime_c_kdl = median(timeit_c_kdl)/nitr*1000000

timeit_id_u2c = repeat("id_u2c_func()", setup = "from __main__ import id_u2c_func", repeat = nrepeat, number = nitr)
mediantime_id_u2c  = median(timeit_id_u2c)/nitr*1000000

timeit_id_pb = repeat("id_pb_func()", setup = "from __main__ import id_pb_func", repeat = nrepeat, number = nitr)
mediantime_id_pb = median(timeit_id_pb)/nitr*1000000

timeit_id_rbdl = repeat("id_rbdl_func()", setup = "from __main__ import id_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_id_rbdl = median(timeit_id_rbdl)/nitr*1000000


timeit_m_u2c = repeat("m_u2c_func()", setup = "from __main__ import m_u2c_func", repeat = nrepeat, number = nitr)
mediantime_m_u2c  = median(timeit_m_u2c)/nitr*1000000

timeit_m_pb = repeat("m_pb_func()", setup = "from __main__ import m_pb_func", repeat = nrepeat, number = nitr)
mediantime_m_pb = median(timeit_m_pb)/nitr*1000000

timeit_m_rbdl = repeat("m_rbdl_func()", setup = "from __main__ import m_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_m_rbdl = median(timeit_m_rbdl)/nitr*1000000

timeit_m_kdl = repeat("m_kdl_func()", setup = "from __main__ import m_kdl_func", repeat = nrepeat, number = nitr)
mediantime_m_kdl = median(timeit_m_kdl)/nitr*1000000


timeit_fdaba_u2c = repeat("aba_u2c_func()", setup = "from __main__ import aba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fdaba_u2c = median(timeit_fdaba_u2c)/nitr*1000000

timeit_fdcrba_u2c = repeat("crba_u2c_func()", setup = "from __main__ import crba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fdcrba_u2c = median(timeit_fdcrba_u2c)/nitr*1000000

timeit_fd_rbdl = repeat("fd_rbdl_func()", setup = "from __main__ import fd_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_fd_rbdl = median(timeit_fd_rbdl)/nitr*1000000


In [5]:
print "\nsnake_g_u2c=", mediantime_g_u2c
print "\nsnake_id_u2c=", mediantime_id_u2c
print "\nsnake_c_u2c=", mediantime_c_u2c
print "\nsnake_m_u2c=", mediantime_m_u2c

print "\nsnake_g_rbdl=", mediantime_g_rbdl
print "\nsnake_id_rbdl=", mediantime_id_rbdl
print "\nsnake_c_rbdl=", mediantime_c_rbdl
print "\nsnake_m_rbdl=", mediantime_m_rbdl

print "\nsnake_g_pb=", mediantime_g_pb
print "\nsnake_id_pb=", mediantime_id_pb
print "\nsnake_c_pb=", mediantime_c_pb
print "\nsnake_m_pb=", mediantime_m_pb

print "\nsnake_g_kdl=", mediantime_g_kdl
print "\nsnake_c_kdl=", mediantime_c_kdl
print "\nsnake_m_kdl=", mediantime_m_kdl

print "\nfdrbdl=", mediantime_fd_rbdl
print "\nfdcrba=", mediantime_fdcrba_u2c 
print "\nfdaba=", mediantime_fdaba_u2c


snake_g_u2c= 22.6119756699

snake_id_u2c= 53.5010099411

snake_c_u2c= 39.0934944153

snake_m_u2c= 23.6259698868

snake_g_rbdl= 13.9629840851

snake_id_rbdl= 28.8770198822

snake_c_rbdl= 20.7278728485

snake_m_rbdl= 13.9125585556

snake_g_pb= 11.4480257034

snake_id_pb= 24.9879360199

snake_c_pb= 18.3044672012

snake_m_pb= 15.7715082169

snake_g_kdl= 22.5989818573

snake_c_kdl= 30.9793949127

snake_m_kdl= 25.3779888153

fdrbdl= 32.3973894119

fdcrba= 68.923830986

fdaba= 54.648399353


In [ ]:

snake_g_u2c= 22.607922554

snake_id_u2c= 53.5010099411

snake_C_u2c= 38.0384922028

snake_M_u2c= 23.862361908

snake_g_rbdl= 14.088511467

snake_id_rbdl= 29.7459363937

snake_C_rbdl= 21.2104320526

snake_M_rbdl= 14.1924619675

snake_g_pb= 11.3346576691

snake_id_pb= 25.2009630203

snake_C_pb= 18.262386322

snake_M_pb= 15.8640146255

snake_g_kdl= 22.5094556808

snake_C_kdl= 31.4583778381

snake_M_kdl= 25.65741539
fdrbdl= 33.0110788345

fdcrba= 68.923830986

fdaba= 54.6615123749


# Derivatives 

## q

In [ ]:
import casadi as cs

qs =cs.SX.sym("qs",njoints)
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
taus =cs.SX.sym("taus",njoints)

c_j = cs.jacobian(c_u2c(qs, qsdot), qs)
g_j = cs.jacobian(g_u2c(qs), qs)
m_j = cs.jacobian(m_u2c(qs), qs)
id_j = cs.jacobian(id_u2c(qs, qsdot, qsddot), qs)
fd_aba_j = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), qs)
fd_crba_j = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), qs)


g_j = cs.Function("g_j", [qs], [g_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
c_j = cs.Function("C_j", [qs, qsdot], [c_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
m_j = cs.Function("M_j", [qs], [m_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
id_j = cs.Function("invd_j", [qs, qsdot, qsddot], [id_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_j = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_j = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [ ]:
def fdcrba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_j(q_none, qdot_none, tau_none)

def fdaba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_j(q_none, qdot_none, tau_none)
    
    
def m_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2   
    m_j(q_none)
    
def g_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_j(q_none)


def c_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    c_j(q_none, qdot_none)
    
def id_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    id_j(q_none, qdot_none, qddot_none)

def aba_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_aba_j(q_none, qdot_none, tau_none)
    
def crba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    fd_aba_j(q_none, qdot_none, tau_none)

In [ ]:
timeit_idj = repeat("id_j_func()", setup = "from __main__ import id_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000

timeit_gj = repeat("g_j_func()", setup = "from __main__ import g_j_func", repeat = nrepeat, number = nitr)
mediantime_gj = median(timeit_gj)/nitr*1000000

timeit_cj = repeat("c_j_func()", setup = "from __main__ import c_j_func", repeat = nrepeat, number = nitr)
mediantime_cj = median(timeit_cj)/nitr*1000000

timeit_mj = repeat("m_j_func()", setup = "from __main__ import m_j_func", repeat = nrepeat, number = nitr)
mediantime_mj = median(timeit_mj)/nitr*1000000

timeit_fdabaj = repeat("aba_j_func()", setup = "from __main__ import aba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_j_func()", setup = "from __main__ import crba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_g_d=", mediantime_gj
print "\nsnake_c_d=", mediantime_cj
print "\nsnake_m_d=", mediantime_mj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj

In [ ]:


n_invdj = invd_j.n_nodes()
n_Cj = C_j.n_nodes()
n_Mj = M_j.n_nodes()
n_gj = g_j.n_nodes()
n_fd_abaj = fd_aba_j.n_nodes()
n_fd_crbaj = fd_crba_j.n_nodes()

print "\nsnake_nodes_id_j=", n_invdj
print "\nsnake_nodes_c_d=", n_Cj
print "\nsnake_nodes_m_d=", n_Mj
print "\nsnake_nodes_g_d=", n_gj
print "\nsnake_nodes_fdaba_d", n_fd_abaj
print "\nsnake_nodes_fdcrba_d =",n_fd_crbaj




snake_id_d= 58.5190057755

snake_g_d= 24.9165296555

snake_C_d= 43.6835289001

snake_M_d= 38.4865999222

snake_fdaba_d= 63.8264417648

snake_fdcrba_d= 69.1299438477

## qdot 

In [ ]:
c_jdot = cs.jacobian(c_u2c(qs, qsdot), qsdot)
id_jdot = cs.jacobian(id_u2c(qs, qsdot, qsddot), qsdot)
fd_aba_jdot = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), qsdot)
fd_crba_jdot = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), qsdot)

c_jdot = cs.Function("c_j", [qs, qsdot], [c_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
id_jdot = cs.Function("id_j", [qs, qsdot, qsddot], [id_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_jdot = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_jdot = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [ ]:
timeit_idj = repeat("id_j_func()", setup = "from __main__ import id_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000


timeit_cj = repeat("c_j_func()", setup = "from __main__ import c_j_func", repeat = nrepeat, number = nitr)
mediantime_cj = median(timeit_cj)/nitr*1000000


timeit_fdabaj = repeat("aba_j_func()", setup = "from __main__ import aba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_j_func()", setup = "from __main__ import crba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_C_d=", mediantime_cj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj

In [ ]:

snake_id_d= 53.680062294

snake_C_d= 36.3329648972

snake_fdaba_d= 57.6950311661

snake_fdcrba_d= 73.1304883957

## qddot / tau

In [ ]:
id_j = cs.jacobian(id_u2c(qs, qsdot, qsddot), qsddot)
fd_aba_j = cs.jacobian(fd_aba_u2c(qs, qsdot, taus), taus)
fd_crba_j = cs.jacobian(fd_crba_u2c(qs, qsdot, taus), taus)

id_j = cs.Function("invd_j", [qs, qsdot, qsddot], [id_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_j = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_j = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [ ]:
timeit_idj = repeat("id_j_func()", setup = "from __main__ import id_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000

timeit_fdabaj = repeat("aba_j_func()", setup = "from __main__ import aba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_j_func()", setup = "from __main__ import crba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj

In [ ]:

snake_id_d= 54.7025203705

snake_fdaba_d= 58.310508728

snake_fdcrba_d= 73.4159946442